In [2]:
require 'btp'


Install the mechanize gem version ~>2.7.5 for using mechanize functions.


true

In [4]:
module BTP
  class Unigram
    def initialize(path)
      @path      = path
      @words     = []
    end

    def parse(summa_data=nil)
      @words = Helper.read_input(summa_data, @path)
      @dataframe = Daru::DataFrame.new({}, order: %i[word_index character left_position right_position count])
      @words.each_with_index { |word, i| unigram_dataframe(word, i) }

      @dataframe
    end

    private

    def unigram_dataframe(word, i)
      word_length = word.length

      word.split('').each_with_index do |char, left|
        df = @dataframe
        bool_array = df[:word_index].eq(i) & df[:character].eq(char) & df[:left_position].eq(left) & df[:right_position].eq(left - word_length)
        df = df.where(bool_array)

        if df.nrows.zero?
          @dataframe.add_row([i, char, left, left - word_length, 1])
        else
          index = df.index.to_a.first
          @dataframe[:count][index] += 1
        end
      end
    end
  end
end
unigram_df2 = BTP::Unigram.new('../../spec/fixtures/finnish/finnish-train-high').parse

,word_index,character,left_position,right_position,count
0,0,t,0,-12,1
1,0,a,1,-11,1
2,0,v,2,-10,1
3,0,a,3,-9,1
4,0,r,4,-8,1
5,0,a,5,-7,1
6,0,m,6,-6,1
7,0,e,7,-5,1
8,0,r,8,-4,1
9,0,k,9,-3,1


In [19]:
vectors = [:character, :left_position, :right_position]
hash = unigram_df2.group_by(vectors).groups.map { |k,v| [k,v.size].flatten }
unigram2_count = Daru::DataFrame.rows(hash, order: vectors+[:count])

,character,left_position,right_position,count
0,,3,-16,1
1,,4,-15,1
2,,5,-12,2
3,,5,-11,1
4,,5,-8,1
5,,5,-6,1
6,,6,-9,1
7,,6,-6,5
8,,6,-5,1
9,,7,-25,1


In [20]:
unigram2_count.sort([:count], ascending: false)

,character,left_position,right_position,count
1526,i,9,-1,198
1542,i,10,-1,147
1511,i,8,-1,143
297,a,9,-1,136
1472,i,6,-4,135
143,a,1,-9,130
1490,i,7,-4,126
1493,i,7,-1,122
311,a,10,-1,115
282,a,8,-1,114


In [2]:
unigram_df = BTP::Unigram.new('../../spec/fixtures/finnish/finnish-task1-train').parse

,character,left_position,right_position,count
0,a,0,-14,32
1,a,1,-13,72
2,l,2,-12,31
3,l,3,-11,25
4,o,4,-10,39
5,i,5,-9,46
6,t,6,-8,25
7,t,7,-7,24
8,a,8,-6,40
9,i,9,-5,102


In [3]:
unigram_df.sort([:count], ascending: false)

,character,left_position,right_position,count
35,i,9,-1,661
88,i,8,-1,548
487,i,10,-1,498
21,i,7,-1,412
30,t,8,-2,332
79,i,11,-1,309
155,t,9,-2,297
452,i,6,-4,275
143,i,12,-1,267
200,i,4,-1,255


In [13]:
chars = unigram_df[:character].uniq.to_a
merged_distribution = Daru::Vector.new(chars.map { |char| [char, unigram_df.where(unigram_df[:character].eq(char))[:count].sum] }.sort_by { |k, _v| k }.to_h)

-,86
a,7047
b,541
c,70
d,959
e,4021
f,497
g,503
h,1352
i,11551
j,564
